In [1]:
import pandas as pd
import numpy as np
import os
from tensorflow.keras import metrics
from tensorflow.keras.layers import Input, LSTM, RepeatVector, Dense, TimeDistributed
# import keras
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import save_model
from util import *
from model import *

In [2]:
es = pd.read_csv("controlled.csv")

print(es.columns)

print(es.head())

Index(['Unnamed: 0', 'Timestamp', 'DateTime', 'X', 'Y', 'Z', 'user', 'user_id',
       'is_controlled', 'label'],
      dtype='object')
   Unnamed: 0      Timestamp               DateTime         X         Y  \
0           0  1550893511286  Feb 22, 2019 22:45:11  0.968506  0.117676   
1           1  1550893511318  Feb 22, 2019 22:45:11  1.016113  0.131592   
2           2  1550893511332  Feb 22, 2019 22:45:11  1.020020  0.100830   
3           3  1550893511365  Feb 22, 2019 22:45:11  0.990479  0.073486   
4           4  1550893511406  Feb 22, 2019 22:45:11  0.946777  0.072266   

          Z    user  user_id is_controlled  label  
0  0.313721  ashwaq        1    controlled  cream  
1  0.382324  ashwaq        1    controlled  cream  
2  0.397705  ashwaq        1    controlled  cream  
3  0.377441  ashwaq        1    controlled  cream  
4  0.355957  ashwaq        1    controlled  cream  


In [3]:
print(np.unique(es['label']))

['blush' 'cream' 'eye' 'gloss' 'mascara']


In [4]:
data_labels = list(np.unique(es['label'])) #['blush', 'cream', 'eye', 'gloss', 'mascara']

num_labels = [0,1,2,3,4]

es['num_label'] = es['label'].apply(data_labels.index)

In [5]:
user_ids = es['user_id']
num_labels= es['num_label']
alph_labels = es['label']
cat_labels = to_categorical(num_labels)
# print(cat_labels.shape)

In [6]:
# es.to_csv('controlled_new.csv')

# Data processing

In [7]:
es= es.drop(['Unnamed: 0', 'DateTime', 'user', 'user_id', 'label', 'num_label', 'is_controlled', 'Timestamp'], axis=1)
es.columns

Index(['X', 'Y', 'Z'], dtype='object')

In [8]:
len(es)

222454

In [9]:
# feats, train_X, train_Y, test_X, test_Y, X = generate_datasets_for_training(es, user_ids, labels, 4)
epochs = 100
batch_size = 32
window_length = 4

In [10]:
es.head()

,X,Y,Z
0,0.968506,0.117676,0.313721
1,1.016113,0.131592,0.382324
2,1.020020,0.100830,0.397705
3,0.990479,0.073486,0.377441
4,0.946777,0.072266,0.355957


In [11]:
# X= np.array(X)
feats, train_X, train_Y, valid_X, valid_Y, test_X, test_Y, X, Y, alph_Y, Z, U = generate_datasets_for_training(es, user_ids, cat_labels, alph_labels, window_length)

In [12]:
print(f'Raw:{es.shape},\nFeatures: {feats}\nTrain: {train_X.shape}\nValid: {valid_X.shape} \nTest: {test_X.shape}\n{np.array(X).shape}')

Raw:(222454, 3),
Features: 3
Train: (125088, 4, 3)
Valid: (41697, 4, 3) 
Test: (55596, 4, 3)
(222381, 4, 3)


In [13]:
print(f'Train labels: {train_Y.shape}\nValid: {valid_Y.shape} \nTest labels: {test_Y.shape}\nCategorical_labels:{np.array(Y).shape} \nAlph labels:{np.array(alph_Y).shape}\nWO block labels:{np.array(Z).shape}')

Train labels: (125088, 5)
Valid: (41697, 5) 
Test labels: (55596, 5)
Categorical_labels:(222381, 4, 5) 
Alph labels:(222381, 4)
WO block labels:(222381, 5)


In [17]:
# raw_labels =[]
# for block in alph_Y:
#     unique_labels=np.unique(block)
# #     print(len(unique_labels), unique_labels, block)
#     if len(unique_labels)==1:
#         raw_labels.append(block)
#     if len(unique_labels)>1:
#         print(len(unique_labels), unique_labels, block)

# Models

In [14]:
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=1e-2, patience=5, verbose=0, mode='auto',
    baseline=None, restore_best_weights=True)

In [14]:
autoencoder_model, encoder_model = get_autoencoder(window_length,feats)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 4, 3)]            0         
_________________________________________________________________
encoder_1 (LSTM)             (None, 4, 128)            67584     
_________________________________________________________________
encoder_2 (LSTM)             (None, 4, 64)             49408     
_________________________________________________________________
encoder_3 (LSTM)             (None, 4, 32)             12416     
_________________________________________________________________
encoder_4 (LSTM)             (None, 16)                3136      
_________________________________________________________________
encoder_decoder_bridge (Repe (None, 4, 16)             0         
_________________________________________________________________
decoder_1 (LSTM)             (None, 4, 16)             2112  

In [15]:
ae_history = train_autoencoder(autoencoder_model, train_X, valid_X, 'controlled', 10, 32)

Epoch 1/10
3909/3909 [==============================] - 95s 23ms/step - loss: 0.0282 - val_loss: 0.0054
Epoch 2/10
3909/3909 [==============================] - 85s 22ms/step - loss: 0.0044 - val_loss: 0.0029
Epoch 3/10
3909/3909 [==============================] - 85s 22ms/step - loss: 0.0027 - val_loss: 0.0025
Epoch 4/10
3909/3909 [==============================] - 85s 22ms/step - loss: 0.0017 - val_loss: 0.0015
Epoch 5/10
3909/3909 [==============================] - 85s 22ms/step - loss: 0.0013 - val_loss: 0.0012
Epoch 6/10
3909/3909 [==============================] - 85s 22ms/step - loss: 0.0011 - val_loss: 0.0012
Epoch 7/10
3909/3909 [==============================] - 85s 22ms/step - loss: 0.0011 - val_loss: 9.9986e-04
Epoch 8/10
3909/3909 [==============================] - 85s 22ms/step - loss: 9.4022e-04 - val_loss: 9.2390e-04
Epoch 9/10
3909/3909 [==============================] - 85s 22ms/step - loss: 9.0723e-04 - val_loss: 0.0019
Epoch 10/10
3909/3909 [=========================

In [17]:
from tensorflow.keras.models import load_model
autoencoder_model = load_model("controlled_autoencoder_model.h5")

In [16]:
num_classes = 5
enc_layer = 4
classifier_model = get_classifier(num_classes, enc_layer, autoencoder_model)

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 4, 3)]            0         
_________________________________________________________________
encoder_1 (LSTM)             (None, 4, 128)            67584     
_________________________________________________________________
encoder_2 (LSTM)             (None, 4, 64)             49408     
_________________________________________________________________
encoder_3 (LSTM)             (None, 4, 32)             12416     
_________________________________________________________________
encoder_4 (LSTM)             (None, 16)                3136      
_________________________________________________________________
fc1 (Dense)                  (None, 256)               4352      
_________________________________________________________________
output (Dense)               (None, 5)                 1285

In [19]:
cl_history = train_classfier(classifier_model, train_X, train_Y, valid_X, valid_Y, 'controlled', 20, 32)

Epoch 1/20
3909/3909 [==============================] - 51s 12ms/step - loss: 0.6138 - val_loss: 0.4966
Epoch 2/20
3909/3909 [==============================] - 46s 12ms/step - loss: 0.4390 - val_loss: 0.4106
Epoch 3/20
3909/3909 [==============================] - 46s 12ms/step - loss: 0.3741 - val_loss: 0.3723
Epoch 4/20
3909/3909 [==============================] - 46s 12ms/step - loss: 0.3360 - val_loss: 0.3361
Epoch 5/20
3909/3909 [==============================] - 46s 12ms/step - loss: 0.3106 - val_loss: 0.3172
Epoch 6/20
3909/3909 [==============================] - 46s 12ms/step - loss: 0.2931 - val_loss: 0.3099
Epoch 7/20
3909/3909 [==============================] - 46s 12ms/step - loss: 0.2786 - val_loss: 0.3005
Epoch 8/20
3909/3909 [==============================] - 46s 12ms/step - loss: 0.2667 - val_loss: 0.2906
Epoch 9/20
3909/3909 [==============================] - 46s 12ms/step - loss: 0.2559 - val_loss: 0.2915
Epoch 10/20
3909/3909 [==============================] - 46s 12m